In [ ]:
def exponential_peak(x, center, amplitude, decay, baseline, floor, ceiling):
    """Symmetric exponential peak function.

    Parameters
    ----------
    x : ndarray
        Independent variable.
    center : int or float
        The center of the peak.
    amplitude : float
        Amplitude parameter.
    decay : float
        Decay parameter.
    baseline : float
        Baseline parameter.
    floor : float
        Minimum value that the result can take.
    ceiling : float
        Maximum value that the result can take.

    Returns
    -------
    y : ndarray

    """

    # locate the index at which to split data into left and right flanks
    ix_split = bisect_right(x, center)

    # compute left flank
    xl = center - x[:ix_split]
    yl = baseline + amplitude * np.exp(-xl / decay)

    # compute right flank
    xr = x[ix_split:] - center
    yr = baseline + amplitude * np.exp(-xr / decay)

    # prepare output
    y = np.concatenate([yl, yr])

    # apply limits
    y = y.clip(floor, ceiling)

    return y


def skewed_exponential_peak(x, center, amplitude, decay, skew, baseline, floor, ceiling):
    """Asymmetric exponential decay peak function.

    Parameters
    ----------
    x : ndarray
        Independent variable.
    center : int or float
        The center of the peak.
    amplitude : float
        Amplitude parameter.
    decay : float
        Decay parameter.
    skew : float
        Skew parameter.
    baseline : float
        Baseline parameter.
    floor : float
        Minimum value that the result can take.
    ceiling : float
        Maximum value that the result can take.

    Returns
    -------
    y : ndarray

    """

    decay_right = 2**(-skew) * decay
    decay_left = 2**skew * decay

    # locate the index at which to split data into left and right flanks
    ix_split = bisect_right(x, center)

    # compute left flank
    xl = center - x[:ix_split]
    yl = baseline + amplitude * np.exp(-xl / decay_left)

    # compute right flank
    xr = x[ix_split:] - center
    yr = baseline + amplitude * np.exp(-xr / decay_right)

    # prepare output
    y = np.concatenate([yl, yr])

    # apply limits
    y = y.clip(floor, ceiling)

    return y


def gaussian_peak(x, center, amplitude, sigma, baseline, floor, ceiling):
    """Gaussian peak function.

    Parameters
    ----------
    x : ndarray
        Independent variable.
    center : int or float
        The center of the peak.
    amplitude : float
        Amplitude parameter.
    sigma : float
        Width parameter.
    baseline : float
        Baseline parameter.
    floor : float
        Minimum value that the result can take.
    ceiling : float
        Maximum value that the result can take.

    Returns
    -------
    y : ndarray

    """

    y = (baseline +
         amplitude *
         np.exp(-(x - center)**2 / (2 * sigma**2)))

    # apply limits
    y = y.clip(floor, ceiling)

    return y


def skewed_gaussian_peak(x, center, amplitude, sigma, skew, baseline, floor, ceiling):
    """Asymmetric Gaussian peak function.

    Parameters
    ----------
    x : ndarray
        Independent variable.
    center : int or float
        The center of the peak.
    amplitude : float
        Amplitude parameter.
    sigma : float
        Width parameter.
    skew : float
        Skew parameter.
    baseline : float
        Baseline parameter.
    floor : float
        Minimum value that the result can take.
    ceiling : float
        Maximum value that the result can take.

    Returns
    -------
    y : ndarray

    """

    sigma_right = 2**(-skew) * sigma
    sigma_left = 2**skew * sigma

    # locate the index at which to split data into left and right flanks
    ix_split = bisect_right(x, center)

    # compute left flank
    xl = center - x[:ix_split]
    yl = (baseline +
          amplitude *
          np.exp(-xl**2 / (2 * sigma_left**2)))

    # compute right flank
    xr = x[ix_split:] - center
    yr = (baseline +
          amplitude *
          np.exp(-xr**2 / (2 * sigma_right**2)))

    # prepare output
    y = np.concatenate([yl, yr])

    # apply limits
    y = y.clip(floor, ceiling)

    return y


def tex_format_lmfit_param(params, name):
    value = params[name].value
    stderr = params[name].stderr
    if stderr:
        relerr = stderr * 100 / value
    else:
        stderr = np.nan
        relerr = np.nan
    return f"${name}={value:.3f} \\pm {stderr:.3f}$ $({relerr:.1f}\\%)$"


@numba.njit
def hampel_filter(x, size, t=3):
    # https://link.springer.com/article/10.1186/s13634-016-0383-6
    # https://towardsdatascience.com/outlier-detection-with-hampel-filter-85ddf523c73d
    
    y = x.copy()
    mad_scale_factor = 1.4826
    
    for i in range(size, len(x) - size):
        # window
        w = x[i - size:i + size]
        # window median
        m = np.median(w)
        # median absolute deviation
        mad = np.median(np.abs(w - m))
        # MAD scale estimate
        s = mad_scale_factor * mad
        # construct response
        if np.abs(x[i] - m) > (t * s):
            y[i] = m
    
    return y


@numba.njit
def recursive_hampel_filter(x, size, t=3):
    # https://link.springer.com/article/10.1186/s13634-016-0383-6
    # https://towardsdatascience.com/outlier-detection-with-hampel-filter-85ddf523c73d
    
    y = x.copy()
    mad_scale_factor = 1.4826
    
    for i in range(size, len(x) - size):
        # window
        w = y[i - size:i + size]
        # window median
        m = np.median(w)
        # median absolute deviation
        mad = np.median(np.abs(w - m))
        # MAD scale estimate
        S = mad_scale_factor * mad
        # construct response
        if np.abs(y[i] - m) > (t * S):
            y[i] = m
    
    return y

